# 9.6 勾配降下法の実装
バッチ勾配降下法（第４章）を用いながら TensorFlow の勾配降下法について見ていく

## 9.6.1 マニュアルの勾配降下法
- tf.random_uniform() 関数は与えられた形状の乱数を格納したテンソルを生成する
- tf.assign() 関数によって $ \theta_i = \theta_{i-1} - \eta\nabla_{\theta}MSE(\theta) $ の式でパラメータを更新する
- 100エポックごとに平均二乗誤差の値をチェック

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

# データの読み込み
housing = fetch_california_housing()
m, n = housing.data.shape

# 前処理で scikit-learn を使って標準化を行う
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

# エポック数と学習率の設定
n_epochs = 1000
learning_rate = 0.01

In [2]:
# 最小二乗法による線形回帰問題の計算グラフの構築
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="pred")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2 / m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

In [3]:
# 計算の実行
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()
    
print("Best theta:")
print(best_theta)

Epoch 0 MSE = 5.077684
Epoch 100 MSE = 0.6258699
Epoch 200 MSE = 0.5609791
Epoch 300 MSE = 0.55120426
Epoch 400 MSE = 0.54517955
Epoch 500 MSE = 0.54064816
Epoch 600 MSE = 0.53716975
Epoch 700 MSE = 0.53448224
Epoch 800 MSE = 0.5323948
Epoch 900 MSE = 0.53076494
Best theta:
[[ 2.0685523 ]
 [ 0.8784914 ]
 [ 0.14207697]
 [-0.3313743 ]
 [ 0.3484965 ]
 [ 0.00334755]
 [-0.04234134]
 [-0.6839516 ]
 [-0.65901357]]


## 9.6.2 自動微分を使った方法
数式微分（symbolic differentiation）なら偏微分方程式は見つけられるが，面倒で間違いやすい上，得られるコードは必ずしも効率的ではない   
　⇛ TensorFlow の自動微分機能を使えば自動的に効率的な方法で勾配計算を行える  
<br/>
**微分計算の方法**
- 数値微分    
　実装は簡単だが正確性に欠ける
- 数式微分   
　正確な計算が可能だが全く別の計算グラフを構築しなければならない
- フォワードモード自動微分    
　二重数（dual number）を使う
- リバースモード自動微分    
　出力次元+1回のトレースで正確な計算が可能  

<br/>
TensorFlow ではリバースモード自動微分が使われている．詳細は付録D（p.509）を参照のこと

In [7]:
tf.reset_default_graph()
# 計算グラフ上で gradients = ・・・ を tf.gradients() を使って書き換えるだけ
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="pred")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

In [9]:
# 計算の実行
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

# マニュアル実装したときとほぼ同じ結果が得られる
print("Best theta:")
print(best_theta)

Epoch 0 MSE = 12.944657
Epoch 100 MSE = 0.8369553
Epoch 200 MSE = 0.6204883
Epoch 300 MSE = 0.5920519
Epoch 400 MSE = 0.57361066
Epoch 500 MSE = 0.5602673
Epoch 600 MSE = 0.5505775
Epoch 700 MSE = 0.5435335
Epoch 800 MSE = 0.5384069
Epoch 900 MSE = 0.53467107
Best theta:
[[ 2.0685523 ]
 [ 0.82096785]
 [ 0.14873374]
 [-0.19006982]
 [ 0.217304  ]
 [ 0.00651637]
 [-0.04167806]
 [-0.6894497 ]
 [-0.6562385 ]]


## 9.6.3 オプティマイザを使うと
TensorFlow には様々なオプティマイザが実装済み

In [10]:
tf.reset_default_graph()
# 計算グラフ上で gradients = ・・・ と training_op = ・・・ を書き換え
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="pred")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

# この先を書き換える
# 単純な勾配降下法
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
# Momentum（こっちのほうが収束が早い）
optimizer = tf.train.MomentumOptimizer(learning_rate=0.01, momentum=0.9)
training_op = optimizer.minimize(mse)

In [11]:
# 計算の実行
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

# マニュアル実装したときとほぼ同じ結果が得られる
print("Best theta:")
print(best_theta)

Epoch 0 MSE = 9.86366
Epoch 100 MSE = 0.5437534
Epoch 200 MSE = 0.52653486
Epoch 300 MSE = 0.52460945
Epoch 400 MSE = 0.5243593
Epoch 500 MSE = 0.5243261
Epoch 600 MSE = 0.5243217
Epoch 700 MSE = 0.5243211
Epoch 800 MSE = 0.524321
Epoch 900 MSE = 0.524321
Best theta:
[[ 2.068558  ]
 [ 0.829636  ]
 [ 0.11875474]
 [-0.26555827]
 [ 0.3057222 ]
 [-0.0045021 ]
 [-0.03932688]
 [-0.8998477 ]
 [-0.8705049 ]]
